In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn import metrics
import hdbscan
import umap
import umap.plot

In [2]:
# df = pd.read_csv('dataset/text_distance/dataset.csv')
df = pd.read_csv('dataset/semantic_similarity/dataset.csv')

In [3]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [4]:
sentences1 = df['Query'].values
sentences2 = df['Key'].unique()

In [5]:
sentences_text = np.append(sentences1, sentences2)

In [6]:
#Compute embedding for both lists
X = model.encode(sentences_text, convert_to_tensor=True)

#### HDBSCAN + UMAP

In [59]:
X_umap = umap.UMAP(n_neighbors=5, n_components=3).fit_transform(X)
clusterer = hdbscan.HDBSCAN(alpha=1.0, min_cluster_size=5)
hdbscan_cluster = clusterer.fit_predict(X_umap)
hdbscan_cluster

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1])

##### silhouette score

In [60]:
hdbscan_silhouette = metrics.silhouette_score(X_umap, hdbscan_cluster, metric='euclidean')
hdbscan_silhouette

0.6631146

In [61]:
n_group = hdbscan_cluster.max()+1
for i in range(n_group):
    print(f'--- group: {i} ---')
    for j, text in enumerate(sentences_text):
        if hdbscan_cluster[j] == i:
            print(f'    {j}: {text}')

--- group: 0 ---
    0: ข้าวโพด
    1: Maize (corn).
    2: ข้าว
    3: Rice.
    12: ธัญพืช
    13: Cereals
--- group: 1 ---
    4: น้ำเลี้ยง (แซป) และสิ่งสกัดจากพืช สารจำพวกเพกติก เกลือของกรดเพกตินิก (เพกติเนต) และเกลือของเพกติก (เพกเตต)  รวมทั้งวุ้นที่ได้จากสาหร่ายทะเล และวุ้นและยางข้นอื่น ๆ ที่ได้จากผลิตภัณฑ์จากพืช จะดัดแปลงหรือไม่ก็ตาม
    5: Vegetable saps and extracts; pectic substances, pectinates and pectates; agar-agar and other mucilages and thickeners, whether or not modified, derived from vegetable products.
    6: น้ำมันมะพร้าว น้ำมันเนื้อในเมล็ดปาล์ม น้ำมันบาบาสสุ และเศษของน้ำมันดังกล่าว จะทำให้บริสุทธิ์หรือไม่ก็ตาม แต่ต้องไม่ดัดแปลงทางเคมี
    7: Coconut (copra), palm kernel or babassu oil and fractions thereof, whether or not refined, but not chemically modified.
    8: ไขมันและน้ำมันชนิดระเหยยากอื่น ๆ ที่ได้จากพืช  (รวมถึงน้ำมันโจโจบา)  และเศษของของดังกล่าว  (นอกจากที่ระบุไว้ตามประเภทที่ 15.07 ถึง 15.14  เช่น               น้ำมันถั่วเหลือง น้ำมันถั่วลิสง น้ำมันมะกอก น

#### DBSCAN + UMAP

In [68]:
X_umap = umap.UMAP(n_neighbors=5, n_components=3).fit_transform(X)

dbscan_cluster = DBSCAN(eps=0.73, min_samples=2).fit(X_umap)
dbscan_cluster.labels_

array([0, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2])

##### silhouette score

In [69]:
dbscan_silhouette = metrics.silhouette_score(X_umap, dbscan_cluster.labels_, metric='euclidean')
dbscan_silhouette

0.6212773

In [70]:
n_group = dbscan_cluster.labels_.max()+1
for i in range(n_group):
    print(f'--- group: {i} ---')
    for j, text in enumerate(sentences_text):
        if dbscan_cluster.labels_[j] == i:
            print(f'    {j}: {text}')

--- group: 0 ---
    0: ข้าวโพด
    1: Maize (corn).
    2: ข้าว
    3: Rice.
    12: ธัญพืช
    13: Cereals
--- group: 1 ---
    4: น้ำเลี้ยง (แซป) และสิ่งสกัดจากพืช สารจำพวกเพกติก เกลือของกรดเพกตินิก (เพกติเนต) และเกลือของเพกติก (เพกเตต)  รวมทั้งวุ้นที่ได้จากสาหร่ายทะเล และวุ้นและยางข้นอื่น ๆ ที่ได้จากผลิตภัณฑ์จากพืช จะดัดแปลงหรือไม่ก็ตาม
    5: Vegetable saps and extracts; pectic substances, pectinates and pectates; agar-agar and other mucilages and thickeners, whether or not modified, derived from vegetable products.
    8: ไขมันและน้ำมันชนิดระเหยยากอื่น ๆ ที่ได้จากพืช  (รวมถึงน้ำมันโจโจบา)  และเศษของของดังกล่าว  (นอกจากที่ระบุไว้ตามประเภทที่ 15.07 ถึง 15.14  เช่น               น้ำมันถั่วเหลือง น้ำมันถั่วลิสง น้ำมันมะกอก น้ำมันปาล์ม น้ำมันที่ได้จากเมล็ดทานตะวัน น้ำมันเมล็ดฝ
    9: Other fixed vegetable fats and oils (including jojoba oil) and their fractions, (other than those specified of heading no.15.07 to 15.14   e.g. : soya bean, ground-nut, olive, palm, sunflower-seed, cotton

In [33]:
# pip install hdbscan

In [71]:

# from sklearn.datasets import load_digits

# digits = load_digits()

# embedding = umap.UMAP().fit_transform(digits.data)

In [72]:
# X_umap = umap.UMAP().fit(X)
# umap.plot.points(X_umap, labels=clustering.labels_)

In [44]:

# clustering = DBSCAN(eps=3, min_samples=2).fit(X_umap)
# clustering.labels_

In [73]:
# X_umap

In [74]:
# embedding

## Kmean

In [123]:
kmeans = KMeans(n_clusters=6, random_state=0, n_init="auto").fit(X)
kmeans.labels_

array([0, 0, 0, 0, 3, 5, 1, 4, 3, 4, 1, 1, 0, 0, 3, 2, 1, 1], dtype=int32)

##### silhouette score

In [124]:
kmean_silhouette = metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
kmean_silhouette

0.12312923

##### cluster result

In [126]:
n_group = kmeans.labels_.max()+1
for i in range(n_group):
    print(f'--- group: {i} ---')
    for j, text in enumerate(sentences_text):
        if kmeans.labels_[j] == i:
            print(f'    {j}: {text}')

--- group: 0 ---
    0: ข้าวโพด
    1: Maize (corn).
    2: ข้าว
    3: Rice.
    12: ธัญพืช
    13: Cereals
--- group: 1 ---
    6: น้ำมันมะพร้าว น้ำมันเนื้อในเมล็ดปาล์ม น้ำมันบาบาสสุ และเศษของน้ำมันดังกล่าว จะทำให้บริสุทธิ์หรือไม่ก็ตาม แต่ต้องไม่ดัดแปลงทางเคมี
    10: ไขมันหรือน้ำมันที่ได้จากสัตว์หรือพืชและเศษของของดังกล่าว  ที่ผ่านกรรมวิธีไฮโดรจีเนชันเพียงบางส่วนหรือทั้งหมด  ที่ผ่านกรรมวิธีอินเตอร์เอสเทอริฟิเคชัน                  รีเอสเทอริฟิเคชันหรือกรรมวิธีอิเลดิไนเซชัน  จะทำให้บริสุทธิ์หรือไม่ก็ตาม แต่ต้องไม่จัดทำ
    11: Animal or vegetable fats and oils and their fractions, partly or wholly hydrogenated, inter-esterified, re-esterified or elaidinised, whether or not refined, but not further prepared.
    16: ไขมันและน้ำมันที่ได้จากสัตว์หรือพืช และผลิตภัณฑ์ที่แยกได้จากไขมันและน้ำมันดังกล่าว ไขมันที่บริโภคซึ่งจัดทำแล้ว ไขที่ได้จากสัตว์หรือพืช
    17: Animal or vegetable fats and oils and their cleavage products ; prepared edible fats ; animal or vegetable waxes
--- group: 2 ---
 

## OPTICS

In [115]:
optics = OPTICS(min_samples=2, eps=20).fit(X)
optics.labels_

array([ 0, -1,  0, -1,  1, -1,  2, -1, -1, -1,  2, -1,  0, -1,  1, -1,  2,
       -1])

##### silhouette score

In [116]:
optics_silhouette = metrics.silhouette_score(X, clustering.labels_, metric='euclidean')
optics_silhouette

-0.0068369284

##### cluster result

In [127]:
n_group = optics.labels_.max()+1
for i in range(n_group):
    print(f'--- group: {i} ---')
    for j, text in enumerate(sentences_text):
        if clustering.labels_[j] == i:
            print(f'    {j}: {text}')


--- group: 0 ---
    0: ข้าวโพด
    2: ข้าว
    12: ธัญพืช
--- group: 1 ---
    4: น้ำเลี้ยง (แซป) และสิ่งสกัดจากพืช สารจำพวกเพกติก เกลือของกรดเพกตินิก (เพกติเนต) และเกลือของเพกติก (เพกเตต)  รวมทั้งวุ้นที่ได้จากสาหร่ายทะเล และวุ้นและยางข้นอื่น ๆ ที่ได้จากผลิตภัณฑ์จากพืช จะดัดแปลงหรือไม่ก็ตาม
    14: ครั่ง รวมทั้งกัม เรซิน น้ำเลี้ยง(แซบ) และสิ่งสกัดอื่นๆ จากพืช
--- group: 2 ---
    6: น้ำมันมะพร้าว น้ำมันเนื้อในเมล็ดปาล์ม น้ำมันบาบาสสุ และเศษของน้ำมันดังกล่าว จะทำให้บริสุทธิ์หรือไม่ก็ตาม แต่ต้องไม่ดัดแปลงทางเคมี
    10: ไขมันหรือน้ำมันที่ได้จากสัตว์หรือพืชและเศษของของดังกล่าว  ที่ผ่านกรรมวิธีไฮโดรจีเนชันเพียงบางส่วนหรือทั้งหมด  ที่ผ่านกรรมวิธีอินเตอร์เอสเทอริฟิเคชัน                  รีเอสเทอริฟิเคชันหรือกรรมวิธีอิเลดิไนเซชัน  จะทำให้บริสุทธิ์หรือไม่ก็ตาม แต่ต้องไม่จัดทำ
    16: ไขมันและน้ำมันที่ได้จากสัตว์หรือพืช และผลิตภัณฑ์ที่แยกได้จากไขมันและน้ำมันดังกล่าว ไขมันที่บริโภคซึ่งจัดทำแล้ว ไขที่ได้จากสัตว์หรือพืช
